In [3]:
pip install --upgrade scikit-image


   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.9 MB 1.3 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.9 MB 1.3 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/12.9 MB 1.3 MB/s eta 0:00:09
   ---- ----------------------------------- 1.6/12.9 MB 1.3 MB/s eta 0:00:09
   ----- ---------------------------------- 1.8/12.9 MB 1.3 MB/s eta 0:00:09
   ------ --------------------------------- 2.1/12.9 MB 1.3 MB/s eta 0:00:09
   ------- -------------------------------- 2.4/12.9 MB 1.3 MB/s eta 0:00:09
   -------- ------------------------------- 2.6/12.9 MB 1.3 MB/s eta 0:00:08
   -------- ------------------------------- 2.9/12.9 MB 1.3 MB/s eta 0:00:08
   --------- ------------------------------ 3.1/12.9 MB 1.3 MB/s eta 0:00:08
   ---------- ------

  You can safely remove it manually.


In [9]:
import os
import numpy as np
import cv2
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import moments_hu
from tqdm import tqdm

# Function to extract color features (mean and std of RGB channels)
def extract_color_features(image):
    mean = np.mean(image, axis=(0, 1))  # Mean per channel
    std = np.std(image, axis=(0, 1))    # Std deviation per channel
    return np.concatenate([mean, std])

# Function to extract texture features using GLCM
def extract_texture_features(image):
    gray_image = rgb2gray(image)  # Convert to grayscale
    gray_image = (gray_image * 255).astype(np.uint8)  # Scale to 8-bit
    
    glcm = graycomatrix(gray_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    
    return np.array([contrast, dissimilarity, homogeneity, energy, correlation])

# Function to extract shape features using Hu Moments
def extract_shape_features(image):
    gray_image = rgb2gray(image)  # Convert to grayscale
    gray_image = (gray_image * 255).astype(np.uint8)  # Convert to 8-bit

    moments = moments_hu(gray_image)
    log_moments = -np.sign(moments) * np.log10(np.abs(moments))  # Log scale for better representation
    return log_moments

# Paths for input dataset and output files
input_dir = "Resized_IMG_CLASSES"
features_output_file = "manual_features.npy"
labels_output_file = "manual_labels.npy"

# Prepare data storage
features_list = []
labels_list = []

# Process images
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue  # Skip non-folder items

    print(f"Extracting features from class: {class_name}")

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_path, img_name)

        # Load image
        image = cv2.imread(img_path)
        if image is None:
            continue  # Skip unreadable images

        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Extract features
        color_features = extract_color_features(image)
        texture_features = extract_texture_features(image)
        shape_features = extract_shape_features(image)

        # Combine all features
        features = np.concatenate([color_features, texture_features, shape_features])
        features_list.append(features)
        labels_list.append(class_name)  # Store label

# Convert lists to NumPy arrays
features_array = np.array(features_list)
labels_array = np.array(labels_list)

# Save as .npy files
np.save(features_output_file, features_array)
np.save(labels_output_file, labels_array)

print(f"Feature extraction complete! Features saved in {features_output_file} and labels in {labels_output_file}")


Extracting features from class: 1. Eczema


Processing 1. Eczema:   1%|          | 8/1000 [00:00<00:12, 77.28it/s]C:\Users\RYZEN\AppData\Local\Temp\ipykernel_2840\1858187837.py:36: RuntimeWarning: divide by zero encountered in log10
  log_moments = -np.sign(moments) * np.log10(np.abs(moments))  # Log scale for better representation
C:\Users\RYZEN\AppData\Local\Temp\ipykernel_2840\1858187837.py:36: RuntimeWarning: invalid value encountered in multiply
  log_moments = -np.sign(moments) * np.log10(np.abs(moments))  # Log scale for better representation
Processing 1. Eczema: 100%|██████████| 1000/1000 [00:14<00:00, 69.85it/s]


Extracting features from class: 2. Melanoma


Processing 2. Melanoma: 100%|██████████| 1000/1000 [00:14<00:00, 70.26it/s]


Extracting features from class: 3. Atopic Dermatitis


...

Extracting features from class: 6. Fungal Infections


Processing 6. Fungal Infections: 100%|██████████| 1000/1000 [00:14<00:00, 68.14it/s]


Extracting features from class: 7. Viral Infections


Processing 7. Viral Infections: 100%|██████████| 1000/1000 [00:13<00:00, 72.54it/s]

Feature extraction complete! Features saved in manual_features.npy and labels in manual_labels.npy
